In [ ]:
import torch
from torchvision import models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the data from the drive:

train_dir = "/content/drive/MyDrive/_Δατα_sсιεисэ/PyTorch_experimenting/data/pizza_steak_sushi"
test_dir = "/content/drive/MyDrive/_Δατα_sсιεисэ/PyTorch_experimenting/data/pizza_steak_sushi"

In [1]:
# Load ResNet50 with V2 pretrained weights
weights_v2 = models.ResNet50_Weights.IMAGENET1K_V2
model_v2 = models.resnet50(weights=weights_v2)

# Get the preprocessing transforms associated with these weights
preprocess_v2 = weights_v2.transforms()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 208MB/s]


### ResnetV2 ###

In [5]:
# Using the preprocessing from ResNet50 V2 for this example
# (If you switch models, swap preprocess_v2 for the other variant.)

train_dataset = datasets.ImageFolder(
    root=train_dir
    , transform=preprocess_v2
)

test_dataset = datasets.ImageFolder(
    root=test_dir
    , transform=preprocess_v2
)

train_loader = DataLoader(
    dataset=train_dataset
    , batch_size=32
    , shuffle=True
    , num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset
    , batch_size=32
    , shuffle=False
    , num_workers=2
)


In [6]:
# ResNetV2 Model, class, device, criterion & optimizer instantiation;

# Number of classes in Pizza–Steak–Sushi
num_classes = len(train_dataset.classes)  # should be 3

# Use ResNet50 V2 backbone
model = model_v2

# Freeze all layers except the final classifier head
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer (ResNet50 uses model.fc)
model.fc = nn.Linear(
      in_features=model.fc.in_features
      , out_features=num_classes
)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
      model.fc.parameters()
      , lr=1e-3
)

In [7]:
# ResNetV2 Training Loop

epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total * 100

    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Acc: {train_acc:.4f}%")

Epoch 1/5 | Loss: 0.6090 | Acc: 70.6667%
Epoch 2/5 | Loss: 0.5499 | Acc: 75.0000%
Epoch 3/5 | Loss: 0.5137 | Acc: 75.3333%
Epoch 4/5 | Loss: 0.4703 | Acc: 75.0000%
Epoch 5/5 | Loss: 0.4465 | Acc: 75.0000%


### ResnetV1 ###

In [2]:
# Load ResNet50 with V1 pretrained weights

weights_v1 = models.ResNet50_Weights.IMAGENET1K_V1
model_v1 = models.resnet50(weights=weights_v1)
preprocess_v1 = weights_v1.transforms()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s]


In [10]:
# ResNetV1 --> need to use preprocess_v1 transform!

train_dataset = datasets.ImageFolder(
    root=train_dir
    , transform=preprocess_v1
)

test_dataset = datasets.ImageFolder(
    root=test_dir
    , transform=preprocess_v1
)

train_loader = DataLoader(
    dataset=train_dataset
    , batch_size=32
    , shuffle=True
    , num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset
    , batch_size=32
    , shuffle=False
    , num_workers=2
)

In [11]:
# ResNetV1 Model, class, device, criterion & optimizer instantiation;

# Number of classes in Pizza–Steak–Sushi
num_classes = len(train_dataset.classes)  # should be 3

# Use ResNet50 V1 backbone
model = model_v1

# Freeze all layers except the final classifier head
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer (ResNet50 uses model.fc)
model.fc = nn.Linear(
      in_features=model.fc.in_features
    , out_features=num_classes
)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
      model.fc.parameters()
      , lr=1e-3
)

In [12]:
# ResNetV1 Training Loop

epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total * 100

    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Acc: {train_acc:.4f}%")

Epoch 1/5 | Loss: 0.7194 | Acc: 60.6667%
Epoch 2/5 | Loss: 0.5501 | Acc: 74.6667%
Epoch 3/5 | Loss: 0.5522 | Acc: 75.6667%
Epoch 4/5 | Loss: 0.5638 | Acc: 77.3333%
Epoch 5/5 | Loss: 0.5075 | Acc: 76.6667%


### EfficientNet_B0 ###

In [3]:
# Load EfficientNet with B0 pretrained weights

weights_eff = models.EfficientNet_B0_Weights.IMAGENET1K_V1
model_eff = models.efficientnet_b0(weights=weights_eff)
preprocess_eff = weights_eff.transforms()

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 171MB/s]


In [13]:
# Use the preprocess transform for EfficientNet_B0 (preprocess_eff)

train_dataset = datasets.ImageFolder(
    root=train_dir
    , transform=preprocess_eff
)

test_dataset = datasets.ImageFolder(
    root=test_dir
    , transform=preprocess_eff
)

train_loader = DataLoader(
    dataset=train_dataset
    , batch_size=32
    , shuffle=True
    , num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset
    , batch_size=32
    , shuffle=False
    , num_workers=2
)

In [15]:


# Number of classes in Pizza–Steak–Sushi
num_classes = len(train_dataset.classes)  # should be 3

# Use ResNet50 V1 backbone
model = model_eff

# Freeze all layers except the final classifier head
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer --> EfficientNet uses model.classifier
model.classifier = nn.Linear(
      in_features=model.classifier[1].in_features
      , out_features=num_classes
)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
      model.classifier.parameters()
      , lr=1e-3
)

In [16]:
# EfficientNet_B0 Training Loop

epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total * 100

    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Acc: {train_acc:.4f}%")

Epoch 1/5 | Loss: 0.6044 | Acc: 71.6667%
Epoch 2/5 | Loss: 0.5457 | Acc: 74.6667%
Epoch 3/5 | Loss: 0.5142 | Acc: 78.0000%
Epoch 4/5 | Loss: 0.4865 | Acc: 78.0000%
Epoch 5/5 | Loss: 0.4642 | Acc: 78.0000%


In [18]:
# ^^^ It looks like EfficientNet did better on the train loop, so running some more epochs to see if the acc scores can get higher! (Note: ended up re-running another 10 epochs, so 25 epochs in total, w/ 93.333% max acc;)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total * 100

    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Acc: {train_acc:.4f}%")

Epoch 1/10 | Loss: 0.3263 | Acc: 87.3333%
Epoch 2/10 | Loss: 0.3091 | Acc: 89.6667%
Epoch 3/10 | Loss: 0.2974 | Acc: 92.0000%
Epoch 4/10 | Loss: 0.3019 | Acc: 87.6667%
Epoch 5/10 | Loss: 0.2821 | Acc: 91.3333%
Epoch 6/10 | Loss: 0.2911 | Acc: 90.6667%
Epoch 7/10 | Loss: 0.2710 | Acc: 91.0000%
Epoch 8/10 | Loss: 0.2714 | Acc: 90.0000%
Epoch 9/10 | Loss: 0.2674 | Acc: 91.0000%
Epoch 10/10 | Loss: 0.2598 | Acc: 93.3333%


In [19]:
# Running Test Loop on EfficientNet

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

test_acc = correct / total * 100
print(f"Test Accuracy: {test_acc:.4f}%")

Test Accuracy: 99.0000%


In [ ]:
"""
You cannot directly use the Google Drive folder link as a file path in Colab. After you mount your Google Drive, you can navigate to the folder in the file browser on the left-hand side of Colab to find its path. It will likely be something like /content/drive/My Drive/Your Folder Name.

Once you have the correct path, you can use it to load your data.

/content/drive/MyDrive/_Δατα_sсιεисэ/PyTorch_experimenting/data/pizza_steak_sushi
"""